In [58]:
import heartpy as hp
import pandas as pd
import numpy as np
import os
import time
from datetime import datetime, timedelta

#import matplotlib.pyplot as plt
#from scipy.signal import cheby2, sosfilt


In [3]:
os.chdir('C:\\Users\\Owner\\Rutgers University\\Michelle Chen - Rutgers_Neuropsych_Lab\\COVID_Fatigue\\RC_award\\Data\\Empatica')
os.getcwd()

'C:\\Users\\Owner\\Rutgers University\\Michelle Chen - Rutgers_Neuropsych_Lab\\COVID_Fatigue\\RC_award\\Data\\Empatica'

In [8]:
def concatenate_lab_data(SubjID):
    wd = os.getcwd() + '\\' + SubjID + '\\' + SubjID + '_baseline\\'
    eda = hp.get_data(wd + 'EDA.csv')
    bvp = hp.get_data(wd + 'BVP.csv')
    acc = hp.get_data(wd + 'ACC.csv')
    temp = hp.get_data(wd + 'TEMP.csv')
    
    init_time = datetime.fromtimestamp(eda[0]) #they all have the same initial time
    
    eda = eda[2:]
    bvp = bvp[2:]
    acc = acc[2:]
    temp = temp[2:]
    
    eda_interval = timedelta(seconds=1/4)
    bvp_interval = timedelta(seconds=1/64)
    acc_interval = timedelta(seconds=1/32)
    temp_interval = timedelta(seconds=1/4)
    
    eda_timestamps = [init_time + i * eda_interval for i in range(len(eda))]
    bvp_timestamps = [init_time + i * bvp_interval for i in range(len(bvp))]
    acc_timestamps = [init_time + i * acc_interval for i in range(len(acc))]
    temp_timestamps = [init_time + i * temp_interval for i in range(len(temp))]
    
    # Create a new DataFrame with the timestamps and the original data columns
    eda_df = pd.DataFrame(data = eda, columns=['EDA (4 Hz)'])
    eda_df['timestamp'] = eda_timestamps

    bvp_df = pd.DataFrame(data = bvp, columns=['BVP (64 Hz)'])
    bvp_df['timestamp'] = bvp_timestamps

    acc_df = pd.DataFrame(data = acc, columns=['ACC X (32 Hz)', 'ACC Y (32 Hz)', 'ACC Z (32 Hz)'])
    acc_df['timestamp'] = acc_timestamps

    temp_df = pd.DataFrame(data = temp, columns=['TEMP (4 Hz)'])
    temp_df['timestamp'] = temp_timestamps
    
    #merge dataframes together, change column order and fill in NA values
    merged_df = pd.merge(bvp_df, eda_df, on='timestamp', how='left').merge(acc_df, on='timestamp', how='left').merge(temp_df, on='timestamp', how='left')
    
    new_cols = ['timestamp','BVP (64 Hz)', 'EDA (4 Hz)','TEMP (4 Hz)','ACC X (32 Hz)','ACC Y (32 Hz)','ACC Z (32 Hz)',]

    merged_df = merged_df[new_cols]
    merged_df = merged_df.ffill(axis=0)
    
    return merged_df

In [9]:
cov4 = concatenate_lab_data('Cov4')
cov7 = concatenate_lab_data('Cov7')
cov8 = concatenate_lab_data('Cov8')
cov13 = concatenate_lab_data('Cov13')
cov14 = concatenate_lab_data('Cov14')
cov20 = concatenate_lab_data('Cov20')

In [10]:
#now we need to add the fatigue ratings
ewd = 'C:\\Users\\Owner\\Rutgers University\\Michelle Chen - Rutgers_Neuropsych_Lab\\COVID_Fatigue\\RC_award\\Data\\E-Prime\\'

def get_ratings(SubjID):
    epr = pd.read_csv(ewd + SubjID + '\\' + SubjID + '_fatigue_ratings.txt', 
                      sep=':', encoding='UTF-16LE', on_bad_lines='skip')
    epr = epr.filter(regex='FatigueRating', axis=0)
    epr = epr.rename(columns={"*** Header Start ***": "Rating"})
  
    clean = []
    for i in range(len(epr)):
        try:
            clean.append(int(pd.Series.tolist(epr.iloc[i])[0]))
        except Exception:
            pass
    
    d = {'Block': [0,1,2,3,4,5,6], 'Rating': clean}
    clean_df = pd.DataFrame(d)
    
    return clean_df

In [60]:
#we also need to extract the tags for each block

def get_tags(SubjID, data):
    wd = os.getcwd() + '\\' + SubjID + '\\' + SubjID + '_baseline\\'
    tags = hp.get_data(wd + 'tags.csv')
    
    if len(tags) != 9:
        print("Incorrect number of tags. There are " + len(tags) + " when there should be 9.")
        return None
    
    init_time= time.mktime(data.loc[0]['timestamp'].timetuple())
    sample_rate = 64

    tag1 = int((tags[0]-init_time)*sample_rate) #number of measurements that are in warmup period, rounded
    baseline = int((tags[1]-init_time)*sample_rate) #end of baseline

    block1_init = int((tags[2]-init_time)*sample_rate) #end of block 1
    block1 = int((tags[3]-init_time)*sample_rate) #end of block 1
    block2 = int((tags[4]-init_time)*sample_rate) #end of block 2
    block3 = int((tags[5]-init_time)*sample_rate) #end of block 3
    block4 = int((tags[6]-init_time)*sample_rate) #end of block 4
    block5 = int((tags[7]-init_time)*sample_rate) #end of block 5
    block6 = int((tags[8]-init_time)*sample_rate) #end of block 6
    
    

In [59]:
get_tags

1677266615.0

In [52]:
get_tags('Cov4', cov4)

TypeError: unsupported operand type(s) for -: 'numpy.float64' and 'Timestamp'

In [15]:
nwd = 'C:\\Users\\Owner\\Rutgers University\\Michelle Chen - Rutgers_Neuropsych_Lab\\COVID_Fatigue\\RC_award\\Data\\Concatenated_Data\\'

cov4.to_csv(nwd + 'Cov4_lab.csv', index=False)
cov7.to_csv(nwd + 'Cov7_lab.csv', index=False)
cov8.to_csv(nwd + 'Cov8_lab.csv', index=False)
cov13.to_csv(nwd + 'Cov13_lab.csv', index=False)
cov14.to_csv(nwd + 'Cov14_lab.csv', index=False)
cov20.to_csv(nwd + 'Cov20_lab.csv', index=False)